In [9]:
import pandas as pd
import statsmodels.api as sm
from statsmodels import regression,stats
import itertools
from pykalman import KalmanFilter
idx = pd.IndexSlice

class MultiFactorModel:
    def __init__(self, price_df, equity_df, benchmark_df, factor_list, universe):
        self.price_df = price_df
        self.equity_df = equity_df
        self.benchmark_df = benchmark_df
        self.factor_list = factor_list
        self.universe = universe
        self.valid_universe = None
        self.subset = None
        
        # z-score in universe
        tmp_equity_df = self.calculate_subset_df(self.equity_df,self.universe)
        tmp_equity_df = tmp_equity_df[factor_list].fillna(0) # TODO: fill or drop??
        self.factor_zscore = (tmp_equity_df - tmp_equity_df.groupby(level='date').mean())/tmp_equity_df.groupby(level='date').std()
        
        # valide_universe
        self.valid_universe = self.calculate_valid_universe()
        self.valid_factor_zscore = self.factor_zscore.loc[idx[:,self.valid_universe],:]


    def calculate_valid_universe(self):
        valid_universe = set(self.universe)

        for date,group in self.factor_zscore.groupby(level=0):
            size = group.shape[0]
            new_set = set(group.loc[idx[:,valid_universe],:].index.get_level_values(1).values)
            if(new_set<valid_universe):
                valid_universe = new_set
        # self.valid_universe = valid_universe        
        return valid_universe

    def calculate_subset_df(self,df,subset=None):
        if(subset!=None):
            return df.loc[idx[:,subset],:]
        else:
            return df.loc[idx[:,self.subset],:]
    def calculate_portfolio_risk(self,hp):
        if(~hasattr(self,'V') or self.V==None):
            self.cross_section_regression()
        sigmap = np.sqrt(np.dot(np.dot(hp.T,self.V),hp))
        MCTR = np.dot(self.V,hp)/sigmap
        return self.V, sigmap, MCTR
        
    def time_series_regression(self):
        ts_factor_zscore = sm.add_constant(self.factor_zscore)
        Y = [self.equity_df.xs(asset,level=1)['return'] for asset in self.valid_universe]
        X = [ts_factor_zscore.xs(asset,level=1)[factor_list+['const']] for asset in self.valid_universe]
        reg_results = [regression.linear_model.OLS(y,x).fit().params for y,x in zip(Y,X) if not(x.empty or y.empty)]
        indices = [asset for y, x, asset in zip(Y, X, self.valid_universe) if not(x.empty or y.empty)]
        ts_result_df = pd.DataFrame(reg_results, index=indices)
        return ts_result_df
    
#     def regression(self,tau):
#         Xtik = self.factor_zscore.loc[idx[:,self.valid_universe],:]
#         n_factors = len(self.factor_list)
#         starting_point = np.zeros(n_factors)
#         Fk = starting_point
#         at = self.benchmark_df['return']
#         rti = self.equity_df.loc[idx[:,self.valid_universe],'return']
        
#         def grad_loss(X,F,r,a,tau):
#             n_factor = len(F)
#             n_time = len(self.equity_df.index.get_level_values(level=0).unique())
#             n_stock = len(self.valid_universe)
#             grad = np.zeros(n_factor)
#             for k in range(n_factor):
#                 gradk = 0
#                 for t in range(n_time):
#                     for i in range(n_stock):
#                         gradk += 2*tau**(n_time-t)*(a[t]+np.dot(X[t,i,:],F[:]) - r[t,i])*X[t,i,k]
#                 grad[k] = gradk
#             return grad
                
        
    
    
#TODO: exponential decay regression, alpha contribution
    def cross_section_regression(self, ndays=22, latest=True):
        valid_factor_zscore = self.factor_zscore.loc[idx[:,self.valid_universe],:]
        if(latest):
            dates = valid_factor_zscore.index.get_level_values(level=0).unique()[-ndays:]
        else:
            dates = valid_factor_zscore.index.get_level_values(level=0).unique()[:ndays]
        valid_factor_zscore = valid_factor_zscore.loc[idx[dates,:],:]
        
        result_list = []
        date_list = []
        for date,group in valid_factor_zscore.groupby(level=0):
            X = sm.add_constant(group.loc[:,self.factor_list])
            y = list(equity_df.loc[(date,list(self.valid_universe)),'return'])          
            results = sm.regression.linear_model.OLS(y,X).fit()
            result_list.append(results.params)
            date_list.append(date)

        cs_result_df = pd.DataFrame(result_list,index=date_list)
        F = np.cov(cs_result_df.iloc[:,1:].T)
        X = self.valid_factor_zscore.groupby(level=1).mean()
        V = np.inner(np.inner(X,F),X)
        self.F = F
        self.X = X
        self.V = V
        
        return cs_result_df
    
    def kalman_filter_calibration(self):
        asset_size = len(self.valid_universe)
        factor_size = len(self.factor_list)
        observation_transition_matrix = []
        for date,group in self.valid_factor_zscore.groupby(level=0):
            exposure_matrix = group.values
#             print(np.shape(exposure_matrix))
#             print(asset_size)
            observation_transition_matrix.append(
                np.concatenate((np.eye(asset_size),exposure_matrix),axis=1).tolist()
            )
        observations = self.equity_df.loc[idx[:,self.valid_universe],'return']
        observation_list = []
        for date,group in observations.groupby(level=0):
            observation_list.append(group.values.tolist())
            
        state_transition_matrix = np.eye(factor_size+asset_size)
        state_covariance_matrix = np.eye(factor_size+asset_size)*0.01
        observation_matrices = observation_transition_matrix
        observation_covariance_matrix = np.eye(asset_size)*0.0
        initial_state_mean = np.zeros(factor_size+asset_size)
        initial_state_covariance = np.eye(factor_size+asset_size)*0.01


        nstate = factor_size+asset_size
        nobs = asset_size
#         print(state_transition_matrix.shape)
#         print(state_covariance_matrix.shape)
#         print(np.shape(observation_matrices))
#         print(observation_covariance_matrix.shape==(nobs,nobs))
#         print(initial_state_mean.shape==(nstate,))
#         print(initial_state_covariance.shape==(nstate,nstate))


        kf = KalmanFilter(transition_matrices=state_transition_matrix,
                 transition_covariance=state_covariance_matrix,
                 observation_matrices=observation_matrices,
                 observation_covariance=observation_covariance_matrix,
                 initial_state_mean=initial_state_mean,
                 initial_state_covariance=initial_state_covariance,
                 n_dim_state=nstate,
                 n_dim_obs=nobs)
        returns = kf.filter(observation_list)
        filtered_state_means = returns[0]
        # a and F
        a_list = []
        F_list = []
        for state in filtered_state_means:
            a = state[:asset_size]
            F = state[asset_size:]
            a_list.append(a)
            F_list.append(F)
            
        a_df = pd.Series(list(itertools.chain(*a_list)),index = self.valid_factor_zscore.index,name='alpha')
        
        F_list_temp = [[F,]*asset_size for F in F_list]
        # subset_factor_zscore.index.get_level_values(0).unique()
        F_list_temp = list(itertools.chain(*F_list_temp))
        F_df = pd.DataFrame(F_list_temp,columns=["{}_F".format(f) for f in factor_list],index=self.valid_factor_zscore.index)
        
        return_df = self.equity_df.loc[idx[:,self.valid_universe],'return']
        close_df = self.equity_df.loc[idx[:,self.valid_universe],'close']
        kf_df = pd.concat([self.valid_factor_zscore,F_df,a_df,return_df,close_df],axis=1)
        self.kf_df = kf_df
        return kf_df

        

In [6]:
from rqdata_utils import *
import pandas as pd
import numpy as np
import scipy as sp
import alphalens as al
from pykalman import KalmanFilter


price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv","cn_instrument_info_2012_2018.csv","cn_equity_daily_2012_2018.csv","sectorCode")
healthcare_universe = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values
benchmark_df = benchmark_reader("cn_SH_healthcare_index_2012_2018.csv")
factor_list = ['market_cap', 'pb_ratio', 'ps_ratio']
universe_list = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values

In [10]:
myModel = MultiFactorModel(price_df, equity_df, benchmark_df, factor_list, universe_list)

/Users/xiayuxuan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:40: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [127]:
factorReturn_df = myModel.cross_section_regression()
print(factorReturn_df.head())
hp = np.ones(len(myModel.valid_universe))
V,sigma, MCTR = myModel.calculate_portfolio_risk(hp)
print("V:\n{}\nsigma\n{}\nMCTR:\n{}".format(V,sigma,MCTR))

               const  market_cap  pb_ratio  ps_ratio
2018-03-27  0.020801   -0.005600 -0.000886 -0.000948
2018-03-28 -0.006505   -0.004983 -0.000603  0.001667
2018-03-29  0.002383   -0.003000 -0.000925  0.001257
2018-03-30  0.014809    0.004416  0.000530  0.001303
2018-04-02 -0.000119   -0.003920 -0.000729  0.000986
V:
[[  3.19844460e-05  -7.42713103e-06   5.18342510e-06 ...,   2.25855232e-06
   -1.12620455e-07  -2.46779972e-05]
 [ -7.42713103e-06   1.60948004e-05  -1.76671909e-05 ...,  -9.67456183e-06
   -8.91835400e-08   1.85188600e-05]
 [  5.18342510e-06  -1.76671909e-05   1.98030463e-05 ...,   1.16950009e-05
   -4.77268788e-07  -1.67809236e-05]
 ..., 
 [  2.25855232e-06  -9.67456183e-06   1.16950009e-05 ...,   1.30630599e-05
   -4.88037744e-06   5.75765299e-06]
 [ -1.12620455e-07  -8.91835400e-08  -4.77268788e-07 ...,  -4.88037744e-06
    3.45766356e-06  -1.09477448e-05]
 [ -2.46779972e-05   1.85188600e-05  -1.67809236e-05 ...,   5.75765299e-06
   -1.09477448e-05   6.49979346e-05]]

In [129]:
factorReturn_df.mean()

const         0.000501
market_cap   -0.000582
pb_ratio      0.000264
ps_ratio      0.000358
dtype: float64

In [130]:
myModel.valid_factor_zscore

market_cap  pb_ratio  ps_ratio
date       order_book_id                                
2012-01-04 000004.XSHE     -0.784393 -0.115692  4.830427
           000028.XSHE     -0.093343 -0.124286 -0.672507
           000150.XSHE     -0.734962 -0.129812  0.443827
           000153.XSHE     -0.669329 -0.128580 -0.607426
           000403.XSHE     -0.872529 -0.132652 -0.722958
           000423.XSHE      2.685384 -0.117587  0.574595
           000513.XSHE     -0.112104 -0.128775 -0.487653
           000518.XSHE     -0.346596 -0.121920  1.369648
           000538.XSHE      3.825501 -0.120392 -0.306006
           000566.XSHE     -0.238486 -0.125659  0.127319
           000590.XSHE     -0.634661 -0.123108 -0.017236
           000597.XSHE     -0.561161 -0.129928 -0.604899
           000623.XSHE      0.954198 -0.129715  1.005393
           000650.XSHE      0.080784 -0.120279 -0.374693
           000661.XSHE     -0.280080 -0.119008 -0.259338
           000739.XSHE     -0.564924 -0.127032 -0.501647
           000756.XSHE     -0.560036 -0.130003 -0.616233
           000766.XSHE     -0.607410 -0.126322  0.804151
           000788.XSHE     -0.335589 -0.124882 -0.438443
           000790.XSHE     -0.482537 -0.121167 -0.015536
           000813.XSHE     -0.500715 -0.119670  0.635287
           000908.XSHE     -0.692716  7.937708  2.577839
           000915.XSHE     -0.595092 -0.124791 -0.369299
           000919.XSHE     -0.362958 -0.129056 -0.481757
           000952.XSHE     -0.652107 -0.128736 -0.143038
           000989.XSHE     -0.480813 -0.128734 -0.352363
           000999.XSHE      1.229004 -0.125949 -0.372860
           600055.XSHG     -0.582462 -0.125691 -0.160978
           600056.XSHG     -0.037443 -0.125372 -0.610603
           600062.XSHG      0.242540 -0.128855 -0.555143
...                              ...       ...       ...
2018-04-27 600129.XSHG     -0.386286 -0.466291 -0.598345
           600161.XSHG     -0.019686  0.416494  0.323530
           600195.XSHG     -0.373145 -0.563027 -0.485857
           600196.XSHG      2.895670 -0.161400 -0.223622
           600201.XSHG      0.113265  0.062088  0.382216
           600211.XSHG     -0.438030 -0.255359  0.088526
           600216.XSHG     -0.167055 -0.694285 -0.470667
           600222.XSHG     -0.595562 -0.689779 -0.471111
           600252.XSHG     -0.208940 -0.555459 -0.129018
           600267.XSHG     -0.150463 -0.595423 -0.553041
           600276.XSHG      8.157888  2.876817  0.918481
           600285.XSHG     -0.482011 -0.472470 -0.402149
           600299.XSHG      0.660679 -0.445147 -0.361248
           600422.XSHG     -0.427116 -0.663731 -0.574736
           600613.XSHG     -0.545685 -0.745020 -0.461527
           600645.XSHG     -0.382377  0.305870  0.260895
           600664.XSHG     -0.176938 -0.662512 -0.572002
           600671.XSHG     -0.617299  5.803844  0.189801
           600721.XSHG     -0.567241 -0.645166  0.216717
           600750.XSHG     -0.414041 -0.501238 -0.267168
           600763.XSHG     -0.158636  2.663023  0.453329
           600767.XSHG     -0.624225  0.944079  0.906025
           600771.XSHG      0.015831  1.538809  0.804394
           600781.XSHG     -0.219611 -0.453423 -0.476747
           600789.XSHG     -0.456632 -0.585305 -0.464537
           600812.XSHG     -0.427730 -0.826791 -0.599529
           600851.XSHG     -0.246236 -0.281208  0.475260
           600867.XSHG      1.147521  1.698297  1.113570
           600896.XSHG     -0.513745 -0.372729  9.328180
           601607.XSHG      1.789230 -0.709421 -0.639660

[96705 rows x 3 columns]

In [17]:
myModel.equity_df.index.get_level_values(level=0)

DatetimeIndex(['2012-01-04', '2012-01-04', '2012-01-04', '2012-01-04',
               '2012-01-04', '2012-01-04', '2012-01-04', '2012-01-04',
               '2012-01-04', '2012-01-04',
               ...
               '2018-04-27', '2018-04-27', '2018-04-27', '2018-04-27',
               '2018-04-27', '2018-04-27', '2018-04-27', '2018-04-27',
               '2018-04-27', '2018-04-27'],
              dtype='datetime64[ns]', name='date', length=3617987, freq=None)

In [90]:
def grad_loss(X,F,r,a,tau,n_factor,n_time,n_stock,n_days=None):
    grad = np.zeros(n_factor)
    for k in range(n_factor):
        gradk = 0
        for t in range(n_time):
            if(n_days!=None and n_time-t>n_days):
                continue
            else:
                damper = tau**(n_time-t)
                for i in range(n_stock):
                    gradk += 2*damper*(a[t]+np.dot(X[t,i,:],F[:]) - r[t,i])*X[t,i,k]
        grad[k] = gradk
    return grad
        
def loss(X,F,r,a,tau,n_factor,n_time,n_stock):
    loss = 0
    for t in range(n_time):
        for i in range(n_stock):
            loss += a[t] -r[t,i]
            loss += np.dot(X[t,i,:],F[:])
            loss = loss**2 * tau**(n_time-t)
    return loss
                

In [38]:
X_df = myModel.factor_zscore.loc[idx[:,myModel.valid_universe],:]
n_factor = len(myModel.factor_list)
n_time = len(myModel.factor_zscore.index.get_level_values(0).unique())
n_stock = len(myModel.valid_universe)
starting_point = np.zeros(n_factors)
F = starting_point
a_df = myModel.benchmark_df['return']
r_df = myModel.equity_df.loc[idx[:,myModel.valid_universe],'return']

In [28]:
X.head()

market_cap  pb_ratio  ps_ratio
date       order_book_id                                
2012-01-04 000004.XSHE     -0.784393 -0.115692  4.830427
           000028.XSHE     -0.093343 -0.124286 -0.672507
           000150.XSHE     -0.734962 -0.129812  0.443827
           000153.XSHE     -0.669329 -0.128580 -0.607426
           000403.XSHE     -0.872529 -0.132652 -0.722958

In [39]:
X = X_df.values.reshape(n_time,n_stock,-1)
a = a_df.values
r = r_df.values.reshape(n_time,n_stock)

In [52]:
myModel.valid_factor_zscore.head()

market_cap  pb_ratio  ps_ratio
date       order_book_id                                
2012-01-04 000004.XSHE     -0.784393 -0.115692  4.830427
           000028.XSHE     -0.093343 -0.124286 -0.672507
           000150.XSHE     -0.734962 -0.129812  0.443827
           000153.XSHE     -0.669329 -0.128580 -0.607426
           000403.XSHE     -0.872529 -0.132652 -0.722958

In [112]:
F = np.zeros(n_factor-1)
for step in range(100):
    sse = loss(X[-100:,:,(0,1)],F,r[-100:,:],a[-100:],1,n_factor-1,100,n_stock)
    F -= grad_loss(X[-100:,:,(0,1)],F,r[-100:,:],a[-100:],1,n_factor-1,n_time=100,n_stock=n_stock)*0.00001
    print(sse,F)

0.0010854417362 [  1.00971943e-04   5.04011777e-05]
0.00107570082835 [  1.79256166e-04   8.63179455e-05]
0.00106803567154 [ 0.00024005  0.00011172]
0.00106199029137 [ 0.00028733  0.00012953]
0.00105721257129 [ 0.00032416  0.00014186]
0.00105342954921 [ 0.00035289  0.00015029]
0.00105042890105 [ 0.00037535  0.00015594]
0.00104804496586 [ 0.00039293  0.00015963]
0.00104614813484 [ 0.00040672  0.00016195]
0.00104463675184 [ 0.00041754  0.00016333]
0.00104343090225 [ 0.00042605  0.00016407]
0.00104246763041 [ 0.00043276  0.00016438]
0.00104169724346 [ 0.00043805  0.00016442]
0.00104108044569 [ 0.00044223  0.00016429]
0.00104058610994 [ 0.00044554  0.00016407]
0.00104018953999 [ 0.00044816  0.00016379]
0.00103987111226 [ 0.00045024  0.0001635 ]
0.00103961521159 [ 0.00045189  0.00016322]
0.00103940939576 [ 0.00045321  0.00016294]
0.00103924373807 [ 0.00045426  0.00016269]
0.0010391103093 [ 0.00045509  0.00016247]
0.0010390027685 [ 0.00045576  0.00016227]
0.00103891603942 [ 0.00045629  0.0001

In [121]:
exposure_df.mean()

const         0.000501
market_cap   -0.000582
pb_ratio      0.000264
ps_ratio      0.000358
dtype: float64

In [126]:
exposure_df

,const,market_cap,pb_ratio,ps_ratio
2018-03-27,0.020801,-0.005600,-0.000886,-0.000948
2018-03-28,-0.006505,-0.004983,-0.000603,0.001667
2018-03-29,0.002383,-0.003000,-0.000925,0.001257
2018-03-30,0.014809,0.004416,0.000530,0.001303
2018-04-02,-0.000119,-0.003920,-0.000729,0.000986
2018-04-03,-0.000223,-0.000844,0.003152,0.003903
2018-04-04,0.012473,-0.002332,-0.003176,-0.004564
2018-04-09,0.001731,-0.001213,-0.003338,0.007950
2018-04-10,0.004090,0.003486,-0.003047,0.009011
2018-04-11,-0.002662,-0.001404,0.002325,0.004528
